# LIBRARY AND DATA IMPORTS

In [1]:
import  os
(os.makedirs('ds_RAJ_GUPTA/csv_files', exist_ok=True))
(os.makedirs('ds_RAJ_GUPTA/outputs', exist_ok=True))

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import scipy.stats as stats
import os
import warnings
warnings.filterwarnings('ignore')




# https://drive.google.com/file/d/1PgQC0tO8XN-wqkNyghWc_-mnrYv_nhSf/view?usp=sharing  - "fear-greed"

# https://drive.google.com/file/d/1IAfLZwu6rJzyWKgBToqwSmmVYU6VbjVs/view?usp=sharing  - "historical"




import gdown

file_id = "1PgQC0tO8XN-wqkNyghWc_-mnrYv_nhSf"  # fear-greed
gdown.download(f"https://drive.google.com/uc?id={file_id}", "ds_RAJ_GUPTA/csv_files/fear_greed_index.csv", quiet=False)

file_id = "1IAfLZwu6rJzyWKgBToqwSmmVYU6VbjVs"  # historical
gdown.download(f"https://drive.google.com/uc?id={file_id}", "ds_RAJ_GUPTA/csv_files/historical_data.csv", quiet=False)






# Load fear & greed data
fear_greed_df = pd.read_csv('ds_RAJ_GUPTA/csv_files/fear_greed_index.csv')

# Load trading data
trading_df = pd.read_csv('ds_RAJ_GUPTA/csv_files/historical_data.csv')


Downloading...
From: https://drive.google.com/uc?id=1PgQC0tO8XN-wqkNyghWc_-mnrYv_nhSf
To: c:\Users\rajgu\programming\ds_RAJ_GUPTA\ds_RAJ_GUPTA\csv_files\fear_greed_index.csv
100%|██████████| 90.8k/90.8k [00:00<00:00, 613kB/s]
Downloading...
From: https://drive.google.com/uc?id=1IAfLZwu6rJzyWKgBToqwSmmVYU6VbjVs
To: c:\Users\rajgu\programming\ds_RAJ_GUPTA\ds_RAJ_GUPTA\csv_files\historical_data.csv
100%|██████████| 47.5M/47.5M [00:02<00:00, 17.0MB/s]


In [3]:
fear_greed_df.describe()

,timestamp,value
count,2.644000e+03,2644.000000
mean,1.631899e+09,46.981089
std,6.597967e+07,21.827680
min,1.517463e+09,5.000000
25%,1.574811e+09,28.000000
50%,1.631900e+09,46.000000
75%,1.688989e+09,66.000000
max,1.746164e+09,95.000000


In [4]:
trading_df.describe()

,Execution Price,Size Tokens,Size USD,Start Position,Closed PnL,Order ID,Fee,Trade ID,Timestamp
count,211224.000000,2.112240e+05,2.112240e+05,2.112240e+05,211224.000000,2.112240e+05,211224.000000,2.112240e+05,2.112240e+05
mean,11414.723350,4.623365e+03,5.639451e+03,-2.994625e+04,48.749001,6.965388e+10,1.163967,5.628549e+14,1.737744e+12
std,29447.654868,1.042729e+05,3.657514e+04,6.738074e+05,919.164828,1.835753e+10,6.758854,3.257565e+14,8.689920e+09
min,0.000005,8.740000e-07,0.000000e+00,-1.433463e+07,-117990.104100,1.732711e+08,-1.175712,0.000000e+00,1.680000e+12
25%,4.854700,2.940000e+00,1.937900e+02,-3.762311e+02,0.000000,5.983853e+10,0.016121,2.810000e+14,1.740000e+12
50%,18.280000,3.200000e+01,5.970450e+02,8.472793e+01,0.000000,7.442939e+10,0.089578,5.620000e+14,1.740000e+12
75%,101.580000,1.879025e+02,2.058960e+03,9.337278e+03,5.792797,8.335543e+10,0.393811,8.460000e+14,1.740000e+12
max,109004.000000,1.582244e+07,3.921431e+06,3.050948e+07,135329.090100,9.014923e+10,837.471593,1.130000e+15,1.750000e+12


# DATA PREPROCESSING

In [5]:

# Process Fear & Greed data
fear_greed_df['date_parsed'] = pd.to_datetime(fear_greed_df['date'])
fear_greed_df['year'] = fear_greed_df['date_parsed'].dt.year
fear_greed_df['month'] = fear_greed_df['date_parsed'].dt.month
fear_greed_df['sentiment_normalized'] = (fear_greed_df['value'] - 50) / 50  # -1 to +1 scale

# Add sentiment momentum
fear_greed_df = fear_greed_df.sort_values('date_parsed').copy()
fear_greed_df['sentiment_change_1d'] = fear_greed_df['value'].diff(1)
fear_greed_df['sentiment_change_7d'] = fear_greed_df['value'].diff(7)
fear_greed_df['sentiment_ma_7d'] = fear_greed_df['value'].rolling(7).mean()
fear_greed_df['sentiment_volatility_30d'] = fear_greed_df['value'].rolling(30).std()

# Process Trading data
trading_df['timestamp_cleaned'] = pd.to_datetime(trading_df['Timestamp IST'],
                                                format='%d-%m-%Y %H:%M', errors='coerce')
trading_df['date'] = trading_df['timestamp_cleaned'].dt.date
trading_df['date_parsed'] = pd.to_datetime(trading_df['date'])
trading_df['hour'] = trading_df['timestamp_cleaned'].dt.hour

# Handle missing values
numeric_columns = ['Closed PnL', 'Size USD', 'Size Tokens', 'Fee']
for col in numeric_columns:
    if col in trading_df.columns:
        trading_df[col] = pd.to_numeric(trading_df[col], errors='coerce').fillna(0)

print("✅ Data preprocessing completed")
print(f"Fear & Greed date range: {fear_greed_df['date'].min()} to {fear_greed_df['date'].max()}")
print(f"Trading data date range: {trading_df['date'].min()} to {trading_df['date'].max()}")

✅ Data preprocessing completed
Fear & Greed date range: 2018-02-01 to 2025-05-02
Trading data date range: 2023-05-01 to 2025-05-01


#FEATURE ENGINEERING

In [6]:
# Create daily trading summaries
daily_trading = trading_df.groupby('date_parsed').agg({
    'Size USD': ['sum', 'mean', 'count', 'std', 'min', 'max'],
    'Closed PnL': ['sum', 'mean', 'std', 'min', 'max'],
    'Fee': ['sum', 'mean'],
    'Size Tokens': ['sum', 'mean'],
    'Account': 'nunique',
    'Coin': 'nunique'
}).round(4)

# Flatten column names
daily_trading.columns = [f"{col[0]}_{col[1]}" for col in daily_trading.columns]
daily_trading = daily_trading.reset_index()

# Rename for clarity
column_mapping = {
    'Size USD_sum': 'daily_volume',
    'Size USD_mean': 'avg_trade_size',
    'Size USD_count': 'trade_count',
    'Size USD_std': 'volume_volatility',
    'Size USD_min': 'min_trade_size',
    'Size USD_max': 'max_trade_size',
    'Closed PnL_sum': 'daily_pnl',
    'Closed PnL_mean': 'avg_pnl_per_trade',
    'Closed PnL_std': 'pnl_volatility',
    'Closed PnL_min': 'min_pnl',
    'Closed PnL_max': 'max_pnl',
    'Fee_sum': 'daily_fees',
    'Fee_mean': 'avg_fee_per_trade',
    'Size Tokens_sum': 'daily_tokens_traded',
    'Size Tokens_mean': 'avg_tokens_per_trade',
    'Account_nunique': 'unique_accounts',
    'Coin_nunique': 'unique_coins'
}

daily_trading = daily_trading.rename(columns=column_mapping)

# Add derived features
daily_trading['pnl_ratio'] = daily_trading['daily_pnl'] / (daily_trading['daily_volume'] + 0.01)
daily_trading['fee_ratio'] = daily_trading['daily_fees'] / (daily_trading['daily_volume'] + 0.01)
daily_trading['avg_pnl_per_dollar'] = daily_trading['daily_pnl'] / (daily_trading['daily_volume'] + 0.01)
daily_trading['volume_efficiency'] = daily_trading['daily_volume'] / daily_trading['trade_count']
daily_trading['pnl_per_trade'] = daily_trading['daily_pnl'] / daily_trading['trade_count']

print(f"✅ Daily trading summaries created: {len(daily_trading)} trading days")
print(f"✅ Features engineered: {len(daily_trading.columns)} metrics per day")

✅ Daily trading summaries created: 480 trading days
✅ Features engineered: 23 metrics per day


#DATA MERGING & CORRELATION

In [7]:
# Merge datasets on date
merged_data = pd.merge(
    daily_trading,
    fear_greed_df[['date_parsed', 'value', 'classification', 'sentiment_normalized',
                   'sentiment_change_1d', 'sentiment_change_7d', 'sentiment_ma_7d',
                   'sentiment_volatility_30d']],
    on='date_parsed',
    how='inner'
)

print(f"✅ Datasets merged successfully: {len(merged_data)} overlapping days")

if len(merged_data) > 0:
    # Calculate correlation matrix
    numeric_cols = merged_data.select_dtypes(include=[np.number]).columns
    correlation_vars = [col for col in numeric_cols if merged_data[col].notna().sum() > 5]

    if len(correlation_vars) >= 2:
        corr_matrix = merged_data[correlation_vars].corr()

        # Focus on sentiment correlations
        sentiment_correlations = corr_matrix['sentiment_normalized'].drop('sentiment_normalized')
        significant_correlations = sentiment_correlations[abs(sentiment_correlations) > 0.1]

        print("\n📊 KEY SENTIMENT CORRELATIONS:")
        print("-" * 30)
        for var, corr in significant_correlations.sort_values(key=abs, ascending=False).head(10).items():
            significance = "***" if abs(corr) > 0.5 else "**" if abs(corr) > 0.3 else "*"
            print(f"{var:<25}: {corr:>6.3f} {significance}")
else:
    print("⚠️  No overlapping dates between datasets")

✅ Datasets merged successfully: 479 overlapping days

📊 KEY SENTIMENT CORRELATIONS:
------------------------------
value                    :  1.000 ***
sentiment_ma_7d          :  0.922 ***
sentiment_volatility_30d : -0.602 ***
sentiment_change_7d      :  0.359 **
unique_accounts          : -0.278 *
daily_volume             : -0.264 *
daily_fees               : -0.261 *
trade_count              : -0.245 *
max_trade_size           : -0.243 *
volume_volatility        : -0.223 *


# SENTIMENT PHASE ANALYSIS

In [8]:
sentiment_analysis = merged_data.groupby('classification').agg({
    'daily_volume': ['count', 'mean', 'std', 'sum'],
    'trade_count': ['mean', 'std', 'sum'],
    'daily_pnl': ['mean', 'std', 'sum', 'min', 'max'],
    'avg_trade_size': ['mean', 'std'],
    'pnl_ratio': ['mean', 'std'],
    'unique_accounts': ['mean', 'sum']
}).round(4)

# Flatten and display
sentiment_summary = {}
for sentiment in merged_data['classification'].unique():
    if pd.notna(sentiment):
        sentiment_data = merged_data[merged_data['classification'] == sentiment]
        sentiment_summary[sentiment] = {
            'days': len(sentiment_data),
            'avg_volume': sentiment_data['daily_volume'].mean(),
            'avg_pnl': sentiment_data['daily_pnl'].mean(),
            'avg_trades': sentiment_data['trade_count'].mean(),
            'total_pnl': sentiment_data['daily_pnl'].sum(),
            'win_rate': len(sentiment_data[sentiment_data['daily_pnl'] > 0]) / len(sentiment_data) if len(sentiment_data) > 0 else 0
        }

# Display sentiment summary
for sentiment, metrics in sentiment_summary.items():
    print(f"\n{sentiment}:")
    print(f"  Trading Days: {metrics['days']}")
    print(f"  Avg Volume: ${metrics['avg_volume']:,.2f}")
    print(f"  Avg Daily P&L: ${metrics['avg_pnl']:,.2f}")
    print(f"  Avg Trades/Day: {metrics['avg_trades']:.1f}")
    print(f"  Win Rate: {metrics['win_rate']:.1%}")

# Statistical significance testing
print("\n2. STATISTICAL SIGNIFICANCE TESTS")
print("-" * 35)

sentiment_groups = list(sentiment_summary.keys())
if len(sentiment_groups) >= 2:
    for metric in ['daily_volume', 'daily_pnl', 'trade_count']:
        if metric in merged_data.columns:
            groups_data = []
            for sentiment in sentiment_groups:
                group_data = merged_data[merged_data['classification'] == sentiment][metric].dropna()
                if len(group_data) > 1:
                    groups_data.append(group_data)

            if len(groups_data) >= 2:
                try:
                    f_stat, p_value = stats.f_oneway(*groups_data)
                    significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
                    print(f"{metric}: F={f_stat:.3f}, p={p_value:.4f} {significance}")
                except:
                    print(f"{metric}: Unable to calculate significance")


Greed:
  Trading Days: 193
  Avg Volume: $1,495,246.09
  Avg Daily P&L: $11,140.57
  Avg Trades/Day: 260.6
  Win Rate: 72.5%

Extreme Greed:
  Trading Days: 114
  Avg Volume: $1,091,799.69
  Avg Daily P&L: $23,817.29
  Avg Trades/Day: 350.8
  Win Rate: 87.7%

Neutral:
  Trading Days: 67
  Avg Volume: $2,690,180.05
  Avg Daily P&L: $19,297.32
  Avg Trades/Day: 562.5
  Win Rate: 67.2%

Fear:
  Trading Days: 91
  Avg Volume: $5,311,261.43
  Avg Daily P&L: $36,891.82
  Avg Trades/Day: 679.5
  Win Rate: 73.6%

Extreme Fear:
  Trading Days: 14
  Avg Volume: $8,177,447.25
  Avg Daily P&L: $52,793.59
  Avg Trades/Day: 1528.6
  Win Rate: 64.3%

2. STATISTICAL SIGNIFICANCE TESTS
-----------------------------------
daily_volume: F=10.903, p=0.0000 ***
daily_pnl: F=2.791, p=0.0259 *
trade_count: F=12.491, p=0.0000 ***


#  INSIGHTS GENERATION

In [9]:
insights = []

# Volume insights
fear_sentiments = ['Fear', 'Extreme Fear']
greed_sentiments = ['Greed', 'Extreme Greed']

fear_data = merged_data[merged_data['classification'].isin(fear_sentiments)]
greed_data = merged_data[merged_data['classification'].isin(greed_sentiments)]

if len(fear_data) > 0 and len(greed_data) > 0:
    fear_volume = fear_data['daily_volume'].mean()
    greed_volume = greed_data['daily_volume'].mean()

    if greed_volume > fear_volume * 1.2:
        volume_ratio = greed_volume / fear_volume
        insights.append(f"VOLUME INSIGHT: {volume_ratio:.2f}x higher trading volume during greed phases")
    elif fear_volume > greed_volume * 1.2:
        volume_ratio = fear_volume / greed_volume
        insights.append(f"VOLUME INSIGHT: {volume_ratio:.2f}x higher trading volume during fear phases")

    # Profitability insights
    fear_pnl = fear_data['daily_pnl'].mean()
    greed_pnl = greed_data['daily_pnl'].mean()

    if fear_pnl > greed_pnl:
        insights.append(f"CONTRARIAN OPPORTUNITY: ${fear_pnl:.2f} avg daily P&L during fear vs ${greed_pnl:.2f} during greed")
    elif greed_pnl > fear_pnl:
        insights.append(f"MOMENTUM STRATEGY: ${greed_pnl:.2f} avg daily P&L during greed vs ${fear_pnl:.2f} during fear")

    # Risk insights
    fear_volatility = fear_data['pnl_volatility'].mean()
    greed_volatility = greed_data['pnl_volatility'].mean()

    if greed_volatility > fear_volatility * 1.3:
        insights.append(f"RISK PATTERN: {greed_volatility/fear_volatility:.2f}x higher P&L volatility during greed periods")

    # Trade frequency insights
    fear_trades = fear_data['trade_count'].mean()
    greed_trades = greed_data['trade_count'].mean()

    if greed_trades > fear_trades * 1.2:
        trade_ratio = greed_trades / fear_trades
        insights.append(f"ACTIVITY PATTERN: {trade_ratio:.2f}x more trades per day during greed phases")

# Correlation insights
if 'sentiment_normalized' in corr_matrix.columns:
    top_correlations = sentiment_correlations.abs().sort_values(ascending=False).head(3)
    for var, abs_corr in top_correlations.items():
        actual_corr = sentiment_correlations[var]
        if abs_corr > 0.3:
            direction = "positively" if actual_corr > 0 else "negatively"
            insights.append(f"CORRELATION: {var} is {direction} correlated with sentiment (r={actual_corr:.3f})")

# Display insights
print(f"\n🎯 GENERATED {len(insights)} KEY INSIGHTS:")
print("=" * 50)
for i, insight in enumerate(insights, 1):
    print(f"{i}. {insight}")




🎯 GENERATED 5 KEY INSIGHTS:
1. VOLUME INSIGHT: 4.23x higher trading volume during fear phases
2. CONTRARIAN OPPORTUNITY: $39012.05 avg daily P&L during fear vs $15847.88 during greed
3. CORRELATION: value is positively correlated with sentiment (r=1.000)
4. CORRELATION: sentiment_ma_7d is positively correlated with sentiment (r=0.922)
5. CORRELATION: sentiment_volatility_30d is negatively correlated with sentiment (r=-0.602)


#  VISUALIZATION GENERATION

In [10]:
try:
    # 1. Sentiment Timeline
    fig, axes = plt.subplots(2, 2, figsize=(20, 12))

    # Main timeline
    axes[0,0].plot(fear_greed_df['date_parsed'], fear_greed_df['value'], linewidth=2, alpha=0.8, color='darkblue')
    axes[0,0].fill_between(fear_greed_df['date_parsed'], fear_greed_df['value'], alpha=0.3, color='lightblue')
    axes[0,0].axhline(y=50, color='gray', linestyle='--', alpha=0.7, label='Neutral')
    axes[0,0].axhline(y=25, color='red', linestyle='--', alpha=0.5, label='Fear Threshold')
    axes[0,0].axhline(y=75, color='green', linestyle='--', alpha=0.5, label='Greed Threshold')
    axes[0,0].set_title('Bitcoin Fear & Greed Index Timeline', fontsize=14, fontweight='bold')
    axes[0,0].set_xlabel('Date')
    axes[0,0].set_ylabel('Fear & Greed Value')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)

    # Sentiment distribution
    sentiment_counts = fear_greed_df['classification'].value_counts()
    colors = ['#ff4444', '#ff9999', '#ffff99', '#99ff99', '#44ff44'][:len(sentiment_counts)]
    axes[0,1].pie(sentiment_counts.values, labels=sentiment_counts.index, autopct='%1.1f%%',
                 colors=colors, startangle=90)
    axes[0,1].set_title('Distribution of Sentiment Classifications', fontsize=14, fontweight='bold')

    # Trading volume by sentiment
    if len(merged_data) > 0:
        sentiment_volume = merged_data.groupby('classification')['daily_volume'].mean()
        axes[1,0].bar(sentiment_volume.index, sentiment_volume.values, alpha=0.7, color='darkgreen')
        axes[1,0].set_title('Average Daily Volume by Sentiment', fontsize=14, fontweight='bold')
        axes[1,0].set_ylabel('Daily Volume (USD)')
        axes[1,0].tick_params(axis='x', rotation=45)
        axes[1,0].grid(True, alpha=0.3)

    # P&L by sentiment
    if len(merged_data) > 0 and 'daily_pnl' in merged_data.columns:
        sentiment_pnl = merged_data.groupby('classification')['daily_pnl'].mean()
        colors_pnl = ['red' if x < 0 else 'green' for x in sentiment_pnl.values]
        axes[1,1].bar(sentiment_pnl.index, sentiment_pnl.values, alpha=0.7, color=colors_pnl)
        axes[1,1].axhline(y=0, color='black', linestyle='-', alpha=0.5)
        axes[1,1].set_title('Average Daily P&L by Sentiment', fontsize=14, fontweight='bold')
        axes[1,1].set_ylabel('Daily P&L (USD)')
        axes[1,1].tick_params(axis='x', rotation=45)
        axes[1,1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('ds_RAJ_GUPTA/outputs/comprehensive_analysis_dashboard.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✅ Saved: comprehensive_analysis_dashboard.png")

    # 2. Correlation Heatmap
    if len(correlation_vars) > 1:
        plt.figure(figsize=(28, 20))
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='RdYlBu_r', center=0,
                   square=True, linewidths=0.5, cbar_kws={"shrink": 0.8}, fmt='.3f')
        plt.title('Correlation Matrix: Trading Metrics vs Market Sentiment', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig('ds_RAJ_GUPTA/outputs/correlation_heatmap.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✅ Saved: correlation_heatmap.png")

    print("✅ Visualizations generated successfully")

except Exception as e:
    print(f"⚠️  Error generating visualizations: {e}")

✅ Saved: comprehensive_analysis_dashboard.png
✅ Saved: correlation_heatmap.png
✅ Visualizations generated successfully


#  CSV DATA EXPORT

In [11]:


# 1. Merged sentiment-trading data
if len(merged_data) > 0:
    merged_export = merged_data.copy()
    merged_export['date'] = merged_export['date_parsed'].dt.strftime('%Y-%m-%d')
    merged_export = merged_export.drop('date_parsed', axis=1)
    merged_export.to_csv('ds_RAJ_GUPTA/csv_files/merged_sentiment_trading.csv', index=False)
    print(f"✅ merged_sentiment_trading.csv ({len(merged_export)} records)")

# 2. Daily trading summary
daily_export = daily_trading.copy()
daily_export['date'] = daily_export['date_parsed'].dt.strftime('%Y-%m-%d')
daily_export = daily_export.drop('date_parsed', axis=1)
daily_export.to_csv('ds_RAJ_GUPTA/csv_files/daily_trading_summary.csv', index=False)
print(f"✅ daily_trading_summary.csv ({len(daily_export)} records)")

# 3. Sentiment performance metrics
if sentiment_summary:
    sentiment_perf_data = []
    for sentiment, metrics in sentiment_summary.items():
        sentiment_perf_data.append({
            'sentiment_classification': sentiment,
            'trading_days': metrics['days'],
            'avg_daily_volume': round(metrics['avg_volume'], 2),
            'avg_daily_pnl': round(metrics['avg_pnl'], 2),
            'total_pnl': round(metrics['total_pnl'], 2),
            'avg_trades_per_day': round(metrics['avg_trades'], 1),
            'win_rate': round(metrics['win_rate'], 3),
            'profitability_rank': 0  # Will be filled below
        })

    sentiment_perf_df = pd.DataFrame(sentiment_perf_data)
    sentiment_perf_df['profitability_rank'] = sentiment_perf_df['avg_daily_pnl'].rank(ascending=False).astype(int)
    sentiment_perf_df.to_csv('ds_RAJ_GUPTA/csv_files/sentiment_performance_metrics.csv', index=False)
    print(f"✅ sentiment_performance_metrics.csv ({len(sentiment_perf_df)} sentiment categories)")

# 4. Correlation results
if len(correlation_vars) > 1:
    correlation_results = []
    for i, var1 in enumerate(correlation_vars):
        for j, var2 in enumerate(correlation_vars):
            if i < j:
                corr_val = corr_matrix.loc[var1, var2]
                correlation_results.append({
                    'variable_1': var1,
                    'variable_2': var2,
                    'correlation_coefficient': round(corr_val, 6),
                    'correlation_strength': 'Strong' if abs(corr_val) > 0.7 else 'Moderate' if abs(corr_val) > 0.3 else 'Weak',
                    'correlation_direction': 'Positive' if corr_val > 0 else 'Negative',
                    'abs_correlation': abs(corr_val)
                })

    correlation_df = pd.DataFrame(correlation_results)
    correlation_df = correlation_df.sort_values('abs_correlation', ascending=False).drop('abs_correlation', axis=1)
    correlation_df.to_csv('ds_RAJ_GUPTA/csv_files/correlation_results.csv', index=False)
    print(f"✅ correlation_results.csv ({len(correlation_df)} correlation pairs)")

✅ merged_sentiment_trading.csv (479 records)
✅ daily_trading_summary.csv (480 records)
✅ sentiment_performance_metrics.csv (5 sentiment categories)
✅ correlation_results.csv (378 correlation pairs)
